In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import env
from wrangle import prepd_data, summarize

from datetime import date

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Acquire

##### Function accessesing db server, queries logs, and cohorts tables from the curriculm_logs db.

In [ ]:
#db access
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

#curriculum_logs db. Queried all tables with a LEFT JOIN.
log_sql = "SELECT *\
              FROM logs\
              LEFT JOIN cohorts ON logs.cohort_id = cohorts.id;"

##### Grabs dataset from get_connection function above and formats it into a pandas dataframe.

In [ ]:
#acquires curriculum_logs dataset
def get_log_data():
    return pd.read_sql(log_sql,get_connection('curriculum_logs'))

##### Assigns the queried datframe as 'df'.

In [ ]:
df = get_log_data()
df.head()

##### I created a dictionary with the class types to merge to the df by 'program id'.

In [ ]:
class_type_dict = {'id':[1, 2, 3, 4],
        'Name':['PHP Full Stack Web Development', 
                'Java Full Stack Web Development', 
                'Data Science', 
                'Front End Web Development'],
        'subdomain':['php','java','ds','fe']}
class_type = pd.DataFrame(class_type_dict)
print (class_type)

##### Merge df and class_type datasets

In [ ]:
df = df.merge(class_type, how='left', left_on='program_id', right_on='id')

In [ ]:
df.head()

##### Cache dataset for accessibility.

In [ ]:
df.to_csv('/Users/hector/codeup-data-science/anomaly-detection-project/log_data.csv')

# Prepare

##### Bring in log_data.csv dataframe for prep

In [ ]:
summarize(df)

In [ ]:
#splits the path into two colums, I will keep 'path_2' and drop 'path'
df['path_2'] = df.path.str.split('/').str[1]

In [ ]:
#Found empty string values in path_2. This code replaces the empty cells with 'NaN'
#in order to drop nulls later.
df['path_2'].replace('', np.nan, inplace=True)

In [ ]:
#merge date and time columns
df['date'] = df['date'] +' '+ df['time']#concat time and date

In [ ]:
#datetime conversion and set index   
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date').sort_index()

In [ ]:
#drop columns. these columns have no use.
df = df.drop(columns=['time', 'id_x', 'slack', 'id_y', 'Name', 'deleted_at', 'path'])

In [ ]:
#rename columns
df = df.rename(columns = {'name':'cohort', 'start_date':'class_start_date', 'end_date':'class_end_date'})    

In [ ]:
df.isnull().sum()

In [ ]:
#drops nulls (rows)
df = df.dropna()

In [ ]:
df.isnull().sum()

In [ ]:
df.path_2.value_counts()

In [ ]:
df.shape

In [ ]:
# take subset where path doesn't end with jpg
df = df[~df.path_2.str.endswith('jpg')]

In [ ]:
df.shape

In [ ]:
# take subset where path doesn't end with jpeg
df = df[~df.path_2.str.endswith('jpeg')]

In [ ]:
df.shape

In [ ]:
# take subset where path doesn't end with svg
df = df[~df.path_2.str.endswith('svg')]

In [ ]:
df.shape

In [ ]:
df.path_2.value_counts()

In [ ]:
df['path_2'] = df.path_2.str.lstrip('(\d)')

In [ ]:
df.path_2.value_counts()

In [ ]:
df.shape

In [ ]:
df['path_2'] = df.path_2.str.strip('123')

In [ ]:
df.shape

In [ ]:
df.path_2.value_counts()

In [ ]:
df.shape

In [ ]:
df['path_2'] = df.path_2.str.lstrip('(-)')

In [ ]:
df.shape

In [ ]:
df.path_2.value_counts()

In [ ]:
#converts dtypes for listed features
convert_dict_int = {'cohort_id':int, 'program_id':int, 'class_start_date':'datetime64[ns]', 'class_end_date':'datetime64[ns]', 'created_at':'datetime64[ns]', 'updated_at':'datetime64[ns]'}
df = df.astype(convert_dict_int)

In [ ]:
df.head()

In [ ]:
df.to_csv('/Users/hector/codeup-data-science/anomaly-detection-project/log_clean.csv')

In [ ]:
df.user_id.value_counts().head()

In [ ]:
df.cohort_id.value_counts().head()

In [ ]:
df.ip.value_counts().head()

In [ ]:
df.ip.value_counts(normalize=True).head()

In [ ]:
def value_counts_and_frequencies(s: pd.Series, dropna=True) -> pd.DataFrame:
    return pd.merge(
        s.value_counts(dropna=False).rename('count'),
        s.value_counts(dropna=False, normalize=True).rename('proba'),
        left_index=True,
        right_index=True,
    )

In [ ]:
ip_df = value_counts_and_frequencies(df.ip)
ip_df.head()

In [ ]:
ip_df['count'].sort_values().tail().plot.barh(figsize=(5, 9))

In [ ]:
lesson_df = value_counts_and_frequencies(df.path_2)
lesson_df['count'].sort_values().tail().plot.barh(figsize=(5, 9))

In [ ]:
cohort_df = value_counts_and_frequencies(df.cohort)
cohort_df['count'].sort_values().tail().plot.barh(figsize=(5, 9))

In [ ]:
user_df = value_counts_and_frequencies(df.user_id)
user_df['count'].sort_values().tail().plot.barh(figsize=(5, 9))

In [ ]:
program_df = value_counts_and_frequencies(df.subdomain)
program_df['count'].sort_values().tail().plot.barh(figsize=(5, 9))

# Explore

1. Discrete data is a numerical type of data that includes whole, concrete numbers with specific and fixed data values determined by counting. 
        - disconnected, separate, and distinct
        - to find relationships, trends, and other concepts
        - In many cases, discrete data can be prefixed with “the number of.”
        - whole, round number that can't be broken down into smaller parts.
        - Discrete data includes discrete variables that are finite, numeric, countable, and non-negative integers (5, 10, 15, and so on).
        - can also be categorical - contain a finite number of data values, such as the gender of a person.
        - Discrete data is distributed discretely in terms of time and space.

2. Continuous data includes complex numbers and varying data values that are measured over a specific time interval.-- refers to change over time
        - unspecified number of possible measurements between two realistic points.
        - all about accuracy
        - data sets often carry decimal points
        - The weight of newborn babies
        - The daily wind speed
        - The temperature of a freezer
        - clocking every runner in the Olympics
        - There are infinite possibilities along this line (for example, 5.77 seconds, 5.772 seconds, 5.7699 seconds, etc.), but every new measurement is always somewhere within the range.
        - continuous data can be either numeric or distributed over date and time.
        - changes over time and can have different values at different time intervals.
        - random variables, which may or may not be whole numbers.
        - Regression analysis is one of the most common types of continuous data analysis.

1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

- which lesson appears the most across cohorts?
- groupby cohort and get a count of lessons (max)...maybe?

In [ ]:
lesson_across_cohorts = (df.sort_values('cohort').path_2.value_counts(normalize=True).rename('cohort_count').reset_index())
lesson_across_cohorts.head()

In [ ]:
lesson_across_cohorts.to_csv('/Users/hector/codeup-data-science/anomaly-detection-project/lesson_across_cohorts.csv')

### Tableau

2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
- Opposite of the above (min)
_ graph all cohortsand show 

In [ ]:
cohorts_to_lesson = (df.sort_values('path_2').cohort.value_counts().rename('path_count').reset_index())
cohorts_to_lesson.head()

In [ ]:
cohorts_to_lesson.to_csv('/Users/hector/codeup-data-science/anomaly-detection-project/cohorts_to_lesson.csv')

### Tableau

3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

- student, active, that do not access the curriculum  (ip address ?)

In [ ]:
student_activity = (df.sort_values('ip').user_id.value_counts().rename('ip_count').reset_index())
student_activity.tail()

In [ ]:
student_activity.to_csv('/Users/hector/codeup-data-science/anomaly-detection-project/student_activity.csv')

### Tableau

4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?



5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?



6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?

In [ ]:
accessed = (df.sort_values('path_2').user_id.value_counts().rename('p2_count').reset_index())
accessed.head()

In [ ]:
accessed.to_csv('/Users/hector/codeup-data-science/anomaly-detection-project/accessed.csv')

### Tableau

7. 

In [ ]:
least_accessed = (df.sort_values('path_2').user_id.value_counts().rename('p2_count').reset_index())
least_accessed .tail()

### Same Tableau as number 3

8. Anything else I should be aware of?

In [ ]:
pages = df['ip'].resample('W').count()
pages.head()

In [ ]:
pages.plot()

In [ ]:
# 7 day EMA
ema_7d = pages.ewm(span=7).mean()
ema_7d.head()

In [ ]:
# 14 days EMA
ema_14d = pages.ewm(span=14).mean()
ema_14d.head()

In [ ]:
# 30 days EMA
ema_30d = pages.ewm(span=30).mean()
ema_30d.head()

In [ ]:
# 90 days EMA
ema_90d = pages.ewm(span=90).mean()
ema_90d.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

ax.plot(pages.index, pages, label='Daily', alpha=.5)

ax.plot(pages.index, ema_7d, label = '7-day EMA')
ax.plot(pages.index, ema_14d, label = '14-day EMA')
ax.plot(pages.index, ema_30d, label = '30-day EMA')
ax.plot(pages.index, ema_90d, label = '30-day EMA')

ax.legend(loc='best')
ax.set_ylabel('IP Activity')

plt.show()


# Bollinger Bands and %b

In [ ]:
# set the window span
span = 30

# compute midband
midband = pages.ewm(span=span).mean()

midband.head()

In [ ]:
# compute exponential stdev
stdev = pages.ewm(span=span).std()

stdev.head()


In [ ]:
# compute upper and lower bands
ub = midband + stdev*3
lb = midband - stdev*3
ub

In [ ]:
lb

In [ ]:
# concatenate ub and lb together into one df, bb
bb = pd.concat([ub, lb], axis=1)

bb.columns = ['ub', 'lb']
bb.head()

In [ ]:
my_df = pd.concat([pages, midband, bb], axis=1)
my_df.columns = ['ip_activity', 'midband', 'ub', 'lb']
my_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

ax.plot(my_df.index, my_df.ip_activity, label='IP. Activity')

ax.plot(my_df.index, my_df.midband, label = '30-day EMA/midband')
ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
ax.plot(my_df.index, my_df.lb, label = 'Lower Band')

ax.legend(loc='best')
ax.set_ylabel('IP Activity')

plt.show()

In [ ]:
# get the last measure of gb
last_measure = my_df.iloc[-1].pages
last_measure


In [ ]:
# get the lower band value on the last date
last_lb = my_df.iloc[-1].lb
last_lb

In [ ]:
# get the upper band value on the last date
last_ub = my_df.iloc[-1].ub
last_ub

In [ ]:
# compute %b
last_pct_b = (last_measure - last_lb)/(last_ub - last_lb)

print('%b for last datapoint: ', round(last_pct_b, 2))

In [ ]:
my_df.tail(3)

In [ ]:
my_df['pct_b'] = (my_df['ip_activity'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])

my_df.head()

In [ ]:
span = 30
weight = 3.5

anomalies = pd.DataFrame()
for u in list(df.user_id.unique()):
    user_df = find_anomalies(df, u, span, weight)
    anomalies = pd.concat([anomalies, user_df], axis=0)
